###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
import tensorflow as tf
from aircraft_models import AirMultinomialTemporalCNN
from tfrecord_dataset import feature_description

In [2]:
# Constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
MFCC_SIZE = 128
BUFFER_SIZE = 1000
BATCH_SIZE = 128

In [3]:
# Dataset location
train_file = 'exports/2020-02-07 01-09-35/train.tfrecord'
test_file = 'exports/2020-02-07 01-09-35/test.tfrecord'

In [4]:
# Parses observation from proto format
def tf_parse_observation(example):
    observation = tf.io.parse_single_example(example, feature_description)
    mfcc = observation['mfcc']
    samples = observation['samples']
    observation['spec'] = tf.reshape(observation['spec'],(mfcc,samples))
    return observation
    
# Converts into correct format for training (input,output) = (spec,label)
def tf_transform_observation(data):
    classes = [b'A320-2xx (CFM56-5)',b'B737-7xx (CF56-7B22-)',b'ERJ190 (CF34-10E)',b'B737-8xx (CF56-7B22+)']
    output = 0
    for i, label in enumerate(classes):
        if data['label'] == label:
            output = i
    return tf.transpose(data['spec']), output

In [5]:
# Creates training data pipeline
train_dataset = tf.data.TFRecordDataset([train_file])
train_dataset = train_dataset.map(tf_parse_observation, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(tf_transform_observation, num_parallel_calls=AUTOTUNE).cache()
observation = next(iter(train_dataset))
time, mfcc = observation[0].shape
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [6]:
# Creates test data pipeline
test_dataset = tf.data.TFRecordDataset([test_file])
test_dataset = test_dataset.map(tf_parse_observation, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(tf_transform_observation, num_parallel_calls=AUTOTUNE).cache()
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [7]:
# Building dataset and model
builder = AirMultinomialTemporalCNN(categories=4)
model = builder.build_model((time,mfcc))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 401, 128)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 51, 16)            32784     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 16)            784       
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 33,636
Trainable params: 33,636
Non-trainable params: 0
_________________________________________________________

In [8]:
# Training model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(train_dataset,epochs=100,validation_data=test_dataset)

Epoch 1/100
7/7 [==============================] - 3s 399ms/step - loss: 3.4919 - accuracy: 0.2717 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
7/7 [==============================] - 0s 41ms/step - loss: 3.0743 - accuracy: 0.3687 - val_loss: 2.9667 - val_accuracy: 0.3529
Epoch 3/100
7/7 [==============================] - 0s 42ms/step - loss: 2.9377 - accuracy: 0.3938 - val_loss: 2.8800 - val_accuracy: 0.3529
Epoch 4/100
7/7 [==============================] - 0s 44ms/step - loss: 2.7906 - accuracy: 0.4589 - val_loss: 2.7960 - val_accuracy: 0.3824
Epoch 5/100
7/7 [==============================] - 0s 42ms/step - loss: 2.6894 - accuracy: 0.4943 - val_loss: 2.8162 - val_accuracy: 0.3235
Epoch 6/100
7/7 [==============================] - 0s 42ms/step - loss: 2.5729 - accuracy: 0.5342 - val_loss: 2.7574 - val_accuracy: 0.3529
Epoch 7/100
7/7 [==============================] - 0s 43ms/step - loss: 2.4624 - accuracy: 0.5822 - val_loss: 2.6648 - val_accuracy: 0.3824
Epoch 8/100

Epoch 60/100
7/7 [==============================] - 0s 42ms/step - loss: 0.4542 - accuracy: 0.9452 - val_loss: 0.6064 - val_accuracy: 0.8529
Epoch 61/100
7/7 [==============================] - 0s 43ms/step - loss: 0.4340 - accuracy: 0.9475 - val_loss: 0.7923 - val_accuracy: 0.8529
Epoch 62/100
7/7 [==============================] - 0s 42ms/step - loss: 0.4416 - accuracy: 0.9475 - val_loss: 0.6317 - val_accuracy: 0.8529
Epoch 63/100
7/7 [==============================] - 0s 42ms/step - loss: 0.4320 - accuracy: 0.9429 - val_loss: 0.6839 - val_accuracy: 0.8235
Epoch 64/100
7/7 [==============================] - 0s 42ms/step - loss: 0.4236 - accuracy: 0.9475 - val_loss: 0.7465 - val_accuracy: 0.8529
Epoch 65/100
7/7 [==============================] - 0s 43ms/step - loss: 0.4195 - accuracy: 0.9532 - val_loss: 1.0852 - val_accuracy: 0.8235
Epoch 66/100
7/7 [==============================] - 0s 42ms/step - loss: 0.4145 - accuracy: 0.9395 - val_loss: 0.6330 - val_accuracy: 0.8824
Epoch 67/100
